## Credits
1. This program uses the data collected at https://ourworldindata.org/covid-vaccinations
1. I got inspiration for doing this after seeing an early version by https://github.com/Ken-Freeman

## Setup
Download the data csv file and put it in the data directory prior to running this program. The included shell scripts download data.

## Design Considerations and Artifacts
1. This notebook creates new variables when that would make no sense in a large data problem.
1. It does this so that we can go back and look at previous values

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from io import StringIO
import os

import logging, sys
# Change this to 'DEBUG' to get raw data samples
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [ ]:
# Function that describes the current state of the interesting fields in a vaccination dataframe
# We cannot put the actual logging statements here because they will not show up in the cells that called these functions
def log_vaccination_stats(banner, target_frame):
    # Display Statistics Mumber of missing total vaccinations values - must run against ungrouped (?) - count() is the number of non-nan
    num_rows=target_frame['location'].size
    s2 = "\nNumber of records: {} daily_vaccinations             with data {} empty {}".format(
        num_rows,
        target_frame['daily_vaccinations'].count(),
        target_frame['daily_vaccinations'].isna().sum()
    )
    s3 = "\nNumber of records: {} daily_vaccinations_per_million with data {} empty {}".format(
        num_rows,
        target_frame['daily_vaccinations_per_million'].count(),
        target_frame['daily_vaccinations_per_million'].isna().sum()
    )
    s4 = "\nNumber of records: {} vaccinated_per_hundred         with data {} empty {}".format(
        num_rows,
        target_frame['people_fully_vaccinated_per_hundred'].count(),
        target_frame['people_fully_vaccinated_per_hundred'].isna().sum()
    )
    s5 = "\nNumber of records: {} total_vaccinations             with data {} empty {}".format(
        num_rows,
        target_frame['total_vaccinations'].count(),
        target_frame['total_vaccinations'].isna().sum()
    )
    return banner+s2+s3+s4+s5

# Display statistics: show another way to display using a list of null and not null
# create a series where the null elements are represented by true and a series where non_null elements are represented
def log_vaccination_stats_2(banner,target_frame):
    s2 ="\nNumber of records: {} daily_vaccinations              with data {} empty {}".format(
        target_frame['location'].size, 
        target_frame[pd.notnull(target_frame['daily_vaccinations'])]['daily_vaccinations'].size , 
        target_frame[pd.isnull(target_frame['daily_vaccinations'])]['daily_vaccinations'].size )
    s3 = "\nNumber of records: {} daily_vaccinations_per_million with data {} empty {}".format(
        target_frame['location'].size, 
        target_frame[pd.notnull(target_frame['daily_vaccinations_per_million'])]['daily_vaccinations_per_million'].size , 
        target_frame[pd.isnull(target_frame['daily_vaccinations_per_million'])]['daily_vaccinations_per_million'].size )
    s4 = "\nNumber of records: {} vaccinated_per_hundred         with data {} empty {}".format(
        target_frame['location'].size, 
        target_frame[pd.notnull(target_frame['people_fully_vaccinated_per_hundred'])]['people_fully_vaccinated_per_hundred'].size , 
        target_frame[pd.isnull(target_frame['people_fully_vaccinated_per_hundred'])]['people_fully_vaccinated_per_hundred'].size )
    s5 = "\nNumber of records: {} total_vaccinations             with data {} empty {}".format(
        target_frame['location'].size,                   
        target_frame[pd.notnull(target_frame['total_vaccinations'])]['total_vaccinations'].size , 
        target_frame[pd.isnull(target_frame['total_vaccinations'])]['total_vaccinations'].size )
    return banner+s2+s3+s4+s5

# logs out the field types in a dataframe                                        
def log_datatypes(banner, target_frame):
    output = StringIO()
    target_frame.info(buf=output)
    return "{}\n{}".format(banner,output.getvalue())

# Sort the data by date mixes by location so lets do ordered sorting on two columns
def log_vaccination_samples(banner, target_frame):
    return "{} \n{}".format(
        banner , 
        target_frame.sort_values(['location','date'], ascending=True)[
            ['location','date','daily_vaccinations','daily_vaccinations_per_million','people_fully_vaccinated_per_hundred','total_vaccinations']])                                         


## Load the Data
1. Requests data set selection: 'world' for all coiuntries or 'state' for US States
1. Load the data
1. Convert the date fields from string to date
1. Remove 2020 data because it is depressing

In [ ]:
logging.info("Local data directory holds: %s", os.listdir("data"))
print('"state" or "world"?')
state_or_world = input()
source_file = "must enter a name"

#import data from csv
if (state_or_world == 'state'):
    df_loaded = pd.read_csv('data/vaccinations_state.csv',parse_dates=True)
elif (state_or_world =='world'):
    df_loaded = pd.read_csv('data/vaccinations_world.csv',parse_dates=True)
else : 
    raise RuntimeError("must enter 'state' or 'world' not what you entered")
    
# Log unique location names
logging.debug("Loaded data for countries\n %s", pd.unique(df_loaded["location"]))

# lets convert the dates from strings to real dates
logging.debug(log_datatypes("Schema prior to date conversion", df_loaded))
df_loaded['date'] = pd.to_datetime(df_loaded['date'])
logging.debug(log_datatypes("Schema after to date conversion", df_loaded))
logging.info(log_vaccination_stats("On load", df_loaded))
df_loaded = df_loaded[(df_loaded['date'].dt.year > 2020)]
logging.info(log_vaccination_stats("After prune 2020", df_loaded))

In [ ]:
#Log the countries with the most data
logging.debug("the number of days for each location %s",df_loaded.location.value_counts())
logging.debug("the number of countries with more than a month worth the data %s",df_loaded.location.value_counts()[lambda day_count: day_count > 30])

## Entities do not supply data every day
1. Make all the countries/states have rows for all days
    1. Fill in missing dates.
1. Do not fill with hard coded numerical the values.  
1. We will interpolate or fill based on the specific requirements for a field

## Fill in the missing rows

In [ ]:
logging.info(log_vaccination_stats("Prior to filling in missing rows/days", df_loaded))

# fill in missing dates https://stackoverflow.com/questions/44978196/pandas-filling-missing-dates-and-values-within-group
# but leave new values as nan so that we can try and extrapolate
# this does not "extrapolate" the missing start/end values - probably should fill with 0 from start to first value
df_filled = df_loaded.set_index(
    ['date', 'location']
).unstack(
    #fill_value=0
).asfreq(
    'D', #fill_value=0
).stack(
    # True would drop all rows where all values are NAN which should never happen with this data because every row has a location name.
    dropna=False
).sort_index(level=1).reset_index()

logging.info(log_vaccination_stats_2("\nAfter to adding missing rows/days",df_filled))
logging.debug(log_vaccination_samples("After fill, Before Interpolation location ",df_filled))

## Fill in missing values for the rows we are interested in
We want to smooth out our graph and eliminate dropout days so countries always have _something_ for each day
1. Basically got rid of the 0/ null problem. It takes whatever value the column held before and just inputs it for the next date.
1. This fills every property/column which may not make sense for some columns.  
1. We should only do the fields that make sense
1. Extrapolate the missing start/end values for text fields
1. We could extrapolate for missing end data but but let's not do that for now https://stackoverflow.com/questions/22491628/extrapolate-values-in-pandas-dataframe


In [ ]:
df_interpolated = df_filled.sort_values(['location','date'])

# interpolate numerical values
df_interpolated['daily_vaccinations'] = df_interpolated.groupby('location').daily_vaccinations.transform(lambda x: x.interpolate())
df_interpolated['daily_vaccinations_per_million'] = df_interpolated.groupby('location').daily_vaccinations_per_million.transform(lambda x: x.interpolate())
df_interpolated['total_vaccinations'] = df_interpolated.groupby('location').total_vaccinations.transform(lambda x: x.interpolate())
df_interpolated['people_fully_vaccinated_per_hundred'] = df_interpolated.groupby('location').people_fully_vaccinated_per_hundred.transform(lambda x: x.interpolate())
df_interpolated['people_vaccinated'] = df_interpolated.groupby('location').people_vaccinated.transform(lambda x: x.interpolate())

# String values , iso code, site, etc are the same for all rows in a location so we ffill and bfill in case early rows are missing values
#df_interpolated['iso_code'] = df_interpolated.groupby('location').iso_code.ffill().bfill()
logging.info("Interpolation finished")

# Show the number of missing total vaccinations values - must run against ungrouped (?)
# count() is the number of non-nan
logging.info(log_vaccination_stats("After data interpolation", df_interpolated))
logging.debug(log_vaccination_samples("After Interpolation: location\n%s", df_interpolated))

# Graph It!
#### TODO
Present more data as bubble sizing

# Vaccinated / 100 over Daily Vaccinations / Million

## log(X),log(y) with vaccinations per hundered Y axis max value 70%

The log(y) axis streteches the early progress and compresses the later gains.
This is useful when in the early stages of the pandemic where the overall vaccination rates are low

In [ ]:
df_interpolated['date_str']=df_interpolated.date.astype(str)
import plotly.express as px
fig = px.scatter(data_frame=df_interpolated, 
                title='Full Vaccinated vs Daily Vaccinations (LOG,LOG)',
                 x="daily_vaccinations_per_million", 
                 y="people_fully_vaccinated_per_hundred", 
                 animation_frame="date_str",
                 animation_group="location", 
                 color='location',
                 hover_name="location", 
                 width=800, 
                 height=600,
                 log_x=True, 
                 log_y=True,
                 range_x=[100+df_interpolated['daily_vaccinations_per_million'].min(),
                          df_interpolated['daily_vaccinations_per_million'].max()],
                 range_y=[0.1,80],
                 labels={"daily_vaccinations_per_million": "Daily Vaccinations / million",
                         "people_fully_vaccinated_per_hundred": "Fully Vaccinated / 100",
                         "location": "Country"
                        }
                )
fig.show()


## log(X),linear(y) with vaccinations per hundered Y axis max value 70%
This makes it more aparant how far we have to go

In [ ]:
df_interpolated['date_str']=df_interpolated.date.astype(str)
import plotly.express as px
fig = px.scatter(data_frame=df_interpolated, 
                 title='Full Vaccinated vs Daily Vaccinations (LOG,LOG)',
                 x="daily_vaccinations_per_million", 
                 y="people_fully_vaccinated_per_hundred", 
                 animation_frame="date_str",
                 animation_group="location", 
                 color='location',
                 hover_name="location", 
                 width=800, 
                 height=600,
                 log_x=True, 
                 log_y=False,
                 range_x=[100+df_interpolated['daily_vaccinations_per_million'].min(),
                          df_interpolated['daily_vaccinations_per_million'].max()],
                 range_y=[0.1,80],
                 labels={"daily_vaccinations_per_million": "Daily Vaccinations / million",
                         "people_fully_vaccinated_per_hundred": "Fully Vaccinated / 100",
                         "location": "Country"
                        }                 
                )

fig.show()


# Vaccinated / 100 over Daily Vaccinations raw number
## log(X),log(y) with vaccinations per hundered Y axis max value 70%
The log(y) axis streteches the early progress and compresses the later gains. 
This is useful when in the early stages of the pandemic where the overall vaccination rates are low

In [ ]:

df_interpolated['date_str']=df_interpolated.date.astype(str)
import plotly.express as px
fig = px.scatter(data_frame=df_interpolated, 
    title='Full Vaccinated vs Daily Vaccinations (LOG,LOG)',
                 x="daily_vaccinations", 
                 y="people_fully_vaccinated_per_hundred", 
                 animation_frame="date_str",
                 animation_group="location", 
                 color='location',
                 hover_name="location", 
                 width=800, 
                 height=600,
                 log_x=True, 
                 log_y=True,
                 range_x=[100+df_interpolated['daily_vaccinations'].min(),
                          df_interpolated['daily_vaccinations'].max()],
                 range_y=[0.1,80],
                 labels={"daily_vaccinations": "Daily Vaccinations - Raw",
                         "people_fully_vaccinated_per_hundred": "Fully Vaccinated / 100",
                         "location": "Country"
                        }                 
                )
fig.show()

## log(X),linear(y) with vaccinations per hundered Y axis max value 70%
This makes it more aparant how far we have to go

In [ ]:
df_interpolated['date_str']=df_interpolated.date.astype(str)
import plotly.express as px
fig = px.scatter(data_frame=df_interpolated, 
                 title='Full Vaccinated vs Daily Vaccinations (LOG,LINEAR)',
                 x="daily_vaccinations", 
                 y="people_fully_vaccinated_per_hundred", 
                 animation_frame="date_str",
                 animation_group="location", 
                 color='location',
                 hover_name="location", 
                 width=800, 
                 height=600,
                 log_x=True, 
                 log_y=False,
                 range_x=[100+df_interpolated['daily_vaccinations'].min(),
                          df_interpolated['daily_vaccinations'].max()],
                 range_y=[0.1,80],
                 labels={"daily_vaccinations": "Daily Vaccinations - Raw",
                         "people_fully_vaccinated_per_hundred": "Fully Vaccinated / 100",
                         "location": "Country"
                        }
                )
fig.show()
